In [36]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
import time
from selenium.webdriver.common.by import By
import re
import requests
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

In [37]:
def get_funding_rate_and_notify(url, line_notify_token):
    # Seleniumの設定
    service = Service(executable_path="chromedriver.exe")
    driver = webdriver.Chrome(service=service)
    driver.get(url)
    time.sleep(2)

    # ページから要素を取得
    obj = driver.find_element(By.CLASS_NAME, "FundingRatesContainer-sc-hzw91n-5")
    print(obj.text)

    # 正規表現を使用して目的の部分を抽出
    matches = re.findall(r'\+([\d.]+)%', obj.text)

    # 抽出された結果を数値化して表示（最初の要素のみ）
    if matches:
        first_numeric_value = float(matches[0])
        print(first_numeric_value)

        # LINEに送信
        if first_numeric_value < 0.01:
            message = f"Funding Rate: {first_numeric_value}%"
            headers = {
                "Authorization": f"Bearer {line_notify_token}",
                "Content-Type": "application/x-www-form-urlencoded"
            }
            payload = {"message": message}
            r = requests.post("https://notify-api.line.me/api/notify", headers=headers, params=payload)
            if r.status_code == 200:
                print("LINEに通知しました。")
            else:
                print("LINEに通知できませんでした。")
        else:
            print("Funding Rateが大きく+の状態です。")
    else:
        print("正規表現による抽出ができませんでした。")

    # ブラウザを閉じる
    driver.quit()

if __name__ == "__main__":
    # LINE Notifyのアクセストークンを指定
    line_notify_token = ""

    # 関数を呼び出して処理を実行
    get_funding_rate_and_notify("https://dex.zeta.markets/trade/SOL-PERP", line_notify_token)

/
0.02806
Funding Rateが大きく+の状態です。


In [ ]:
#lambda
#再読み込み

In [ ]:
def lambda_handler(event, context):
    # Seleniumの設定
    url = "https://dex.zeta.markets/trade/SOL-PERP"
    line_notify_token = ""

    options = webdriver.ChromeOptions()
    options.add_argument("--headless")
    options.add_argument("--disable-gpu")
    options.add_argument("--hide-scrollbars")
    options.add_argument("--single-process")
    options.add_argument("--ignore-certificate-errors")
    options.add_argument("--window-size=880x996")
    options.add_argument("--no-sandbox")
    options.add_argument("--homedir=/tmp")
    options.binary_location = "/opt/headless/python/bin/headless-chromium"

    # ブラウザの定義
    driver = webdriver.Chrome("/opt/headless/python/bin/chromedriver", options=options)

    # getメソッドを使用してURLにアクセス
    driver.get(url)

    # ページを再読み込み
    driver.refresh()

    # 要素が表示されるまで待機 (waiting for visibility)
    wait = WebDriverWait(driver, 20)
    obj = wait.until(EC.visibility_of_element_located((By.CLASS_NAME, "FundingRatesContainer-sc-hzw91n-5")))

    # ページから要素を取得
    print(obj.text)

    # 正規表現を使用して目的の部分を抽出
    matches = re.findall(r'\+([\d.]+)%', obj.text)

    # 抽出された結果を数値化して表示（最初の要素のみ）
    if matches:
        first_numeric_value = float(matches[0])
        print(first_numeric_value)

        # LINEに送信
        if first_numeric_value < 0.01:
            message = f"Funding Rate: {first_numeric_value}%"
            headers = {
                "Authorization": f"Bearer {line_notify_token}",
                "Content-Type": "application/x-www-form-urlencoded"
            }
            payload = {"message": message}
            r = requests.post("https://notify-api.line.me/api/notify", headers=headers, params=payload)
            if r.status_code == 200:
                print("LINEに通知しました。")
            else:
                print("LINEに通知できませんでした。")
        else:
            print("Funding Rateが大きく+の状態です。")
    else:
        print("正規表現による抽出ができませんでした。")

    # ブラウザを閉じる
    driver.quit()